# PyLoan, a basic loan simulation notebook

In [1]:
from datetime import datetime, timedelta

%matplotlib widget
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from dateutil import relativedelta
from IPython.display import display
from ipywidgets import Box, Button, HBox, Layout, VBox

plt.ioff()

HLayout = Layout(
    overflow="scroll hidden",
    border="3px solid black",
    width="100%",
    height="100%",
    flex_flow="row",
    display="flex",
)


def mortgage_payments(r, P, n):
    return r * P / (1 - (1 + r) ** -n)


def monthly_payment(rate, amount, years):
    return mortgage_payments(r=rate, P=amount, n=years) / 12


def timeline(start_date, rate, amount, years):
    per_month = monthly_payment(rate, amount, years)
    losses = [0.0]
    times = [start_date - relativedelta.relativedelta(months=1)]
    reminders = [amount]
    for year in range(int(years)):
        yearly_loss = reminders[-1] * rate
        monthly_loss = yearly_loss / 12
        for month in range(12):
            date = start_date + relativedelta.relativedelta(months=month, years=year)
            times.append(date)
            losses.append(losses[-1] + monthly_loss)
            reminders.append(reminders[-1] - (per_month - monthly_loss))
    return per_month, times, losses, reminders


class LoanSim:
    def __init__(self, display_widget=True):
        self.loan_duration_years_w = widgets.IntSlider(min=5, max=25)
        self.loan_sum_w = widgets.BoundedIntText(min=50, max=300)
        self.loan_rate_w = widgets.BoundedFloatText(min=0.01, max=100.0, value=3.0)
        self.loan_start_date_w = widgets.DatePicker(
            disabled=False, value=datetime.now()
        )

        self.loan_total_cost_w = widgets.Label()
        self.loan_per_month_payment_w = widgets.Label()
        self.figure = plt.figure()
        self.figure.canvas.header_visible = False

        VLayout = Layout(
            display="flex", flex_flow="column", align_items="stretch",border="3px solid black", width="50%"
        )
        self.layout = Box(
            children=[
                HBox(
                    [
                        widgets.Label("Start Date:"),
                        self.loan_start_date_w,
                        widgets.Label("Loan term (years):"),
                        self.loan_duration_years_w,
                    ]
                ),
                HBox(
                    [
                        widgets.Label("Amount (k€):"),
                        self.loan_sum_w,
                        widgets.Label("Annual Percentage Rate (%):"),
                        self.loan_rate_w,
                    ]
                ),
                self.loan_total_cost_w,
                self.loan_per_month_payment_w,
                self.figure.canvas,
            ],
            layout=VLayout,
        )
        if display_widget:
            display(self.layout)

        self.losses_line = None
        self.reminders_line = None

        self.update_plot()

        self.loan_start_date_w.observe(self.update_plot, names="value")
        self.loan_duration_years_w.observe(self.update_plot, names="value")
        self.loan_sum_w.observe(self.update_plot, names="value")
        self.loan_rate_w.observe(self.update_plot, names="value")

    def update_plot(self, *args, **kwargs):
        start_date = self.loan_start_date_w.value
        rate = self.loan_rate_w.value / 100
        loan_sum = self.loan_sum_w.value * 1000
        duration_years = self.loan_duration_years_w.value
        per_month, times, losses, reminders = timeline(
            start_date, rate, loan_sum, duration_years
        )
        self.loan_total_cost_w.value = f"Total loan cost: {losses[-1]/1000:.2f}k€"
        self.loan_per_month_payment_w.value = f"Monthly payments: {per_month:.1f}€"
        if self.losses_line is None:
            plt.figure(num=self.figure.number)
            (self.losses_line,) = plt.plot(times, losses, label="Loss")
            (self.reminders_line,) = plt.plot(times, reminders, label="Reminder")
            plt.legend()
        else:
            plt.figure(num=self.figure.number)
            self.losses_line.set_ydata(losses)
            self.losses_line.set_xdata(times)
            self.reminders_line.set_ydata(reminders)
            self.reminders_line.set_xdata(times)
            plt.ylim(0.0, loan_sum)
            plt.xlim(
                start_date,
                start_date + relativedelta.relativedelta(years=duration_years),
            )
            self.figure.canvas.draw()

In [2]:
left, right = LoanSim(display_widget=False), LoanSim(display_widget=False)
display(Box(children=[left.layout, right.layout], layout=HLayout))

Box(children=(Box(children=(HBox(children=(Label(value='Start Date:'), DatePicker(value=datetime.datetime(2022…

In [3]:
left, right = LoanSim(display_widget=False), LoanSim(display_widget=False)
display(Box(children=[left.layout, right.layout], layout=HLayout))

Box(children=(Box(children=(HBox(children=(Label(value='Start Date:'), DatePicker(value=datetime.datetime(2022…